In [77]:
!pip install plotly
!pip install lightgbm
!pip install --upgrade pip
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from IPython.display import Image, display
from pandas.tools.plotting import scatter_matrix 
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn import preprocessing
import lightgbm as lgb 
import datetime

Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (18.1)


In [89]:
train_df = pd.read_csv("/Users/sandy/Documents/ENPM808/Project/Github/google-revenue-prediction/train.csv")
train_df["date"] = pd.to_datetime(train_df["date"], format="%Y%m%d")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [90]:
train_df.head()

,channelGrouping,date,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime
0,Organic Search,2016-09-02,Firefox,desktop,False,Windows,9499785259412240342,not available in demo dataset,Europe,Netherlands,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472812272,1.0,1.472812e+09
1,Organic Search,2016-09-02,Chrome,desktop,False,Windows,3696906537737368442,Buenos Aires,Americas,Argentina,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472856874,1.0,1.472857e+09
2,Organic Search,2016-09-02,Chrome,desktop,False,Linux,5688512451361930466,Bengaluru,Asia,India,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472811524,1.0,1.472812e+09
3,Organic Search,2016-09-02,Safari,mobile,True,iOS,9716453263501147451,New Delhi,Asia,India,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472823508,1.0,1.472824e+09
4,Organic Search,2016-09-02,Chrome,desktop,False,Windows,5330453591145100432,Sao Paulo,Americas,Brazil,...,NaN,(not set),NaN,(not provided),organic,NaN,google,1472839619,1.0,1.472840e+09


In [91]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
#test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float 
# scikit.rf needs numerical data. One hot encoding is not good on rf.
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')))
    
    #lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    #test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    #test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 
dev_df = train_df[train_df['date']<=datetime.date(2017,5,31)]
val_df = train_df[train_df['date']>datetime.date(2017,5,31)]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)


dev_X.loc[:, dev_X.columns != 'totals.transactionRevenue']
val_X.loc[:, val_X.columns != 'totals.transactionRevenue']

#dev_X = dev_df[cat_cols + num_cols] 
#val_X = val_df[cat_cols + num_cols] 

#test_X = test_df[cat_cols + num_cols] 

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.


,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,...,trafficSource.referralPath,trafficSource.source,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect,totals.hits,totals.pageviews,visitNumber,visitStartTime,totals.bounces,totals.newVisits
436393,4,43,0,20,955,4,75,122,0,482,...,3196,99,1,1,1.0,1.0,1.0,1.508198e+09,1.0,1.0
436394,6,35,0,3,220,2,218,100,0,73,...,300,301,1,1,2.0,2.0,6.0,1.508176e+09,0.0,0.0
436395,2,35,1,1,955,2,218,122,40939,482,...,3196,0,1,0,2.0,2.0,1.0,1.508202e+09,0.0,1.0
436396,4,35,0,20,955,3,210,122,38725,482,...,3196,99,1,1,2.0,2.0,1.0,1.508170e+09,0.0,1.0
436397,4,35,0,20,955,2,132,122,29216,482,...,3196,99,1,1,2.0,2.0,1.0,1.508191e+09,0.0,1.0
436398,6,35,0,7,749,2,218,100,38725,73,...,2254,0,1,1,2.0,2.0,1.0,1.508197e+09,0.0,1.0
436399,6,35,0,7,458,4,217,60,0,131,...,249,301,1,1,2.0,2.0,1.0,1.508152e+09,0.0,1.0
436400,4,35,0,20,955,4,55,122,13584,482,...,3196,99,1,1,2.0,2.0,1.0,1.508206e+09,0.0,1.0
436401,4,35,0,7,515,2,132,0,38486,253,...,3196,99,1,1,2.0,2.0,1.0,1.508208e+09,0.0,1.0
436402,4,72,1,23,955,4,144,122,0,482,...,3196,99,1,0,2.0,2.0,2.0,1.508165e+09,0.0,0.0


In [92]:
#NaNs return error in random forest.
dev_X.fillna(0, inplace=True)
val_X.fillna(0, inplace=True)

In [93]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    #pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return model, pred_val_y

# Training the model #
#pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)
model, pred_val_lgb = run_lgb(dev_X, dev_y, val_X, val_y)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.46691
Early stopping, best iteration is:
[68]	valid_0's rmse: 1.46593


In [94]:
#Calculate RMSE
from sklearn import metrics
pred_val_lgb[pred_val_lgb<0] = 0
val_pred_lgb = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_lgb["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_lgb["PredictedRevenue"] = np.expm1(pred_val_lgb)
#print(np.sqrt(metrics.mean_squared_error(np.pred_val_lgb(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_lgb = val_pred_lgb.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_lgb["transactionRevenue"].values), np.log1p(val_pred_lgb["PredictedRevenue"].values))))

1.5014124368310202


In [99]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees


def run_rf(train_X, train_y, val_X, val_y):
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
    rf.fit(train_X, train_y);
    
    pred_val_y = rf.predict(val_X)
    return rf, pred_val_y
# Train the model on training data
rf, pred_val_rf = run_rf(dev_X, dev_y, val_X, val_y);

In [101]:
from sklearn import metrics
pred_val_rf[pred_val_rf<0] = 0
val_pred_rf = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_rf["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_rf["PredictedRevenue"] = np.expm1(pred_val_rf)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_rf = val_pred_rf.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_rf["transactionRevenue"].values), np.log1p(val_pred_rf["PredictedRevenue"].values))))

1.6530850039533813
